In [1]:
import os
from pathlib import Path
import laspy
import numpy as np
from sklearn.cluster import DBSCAN

In [9]:
def run_dbscan(pathin, pathout, minpoints, r):
    """
    Runs DBSCAN clustering on the input LAS file, but only for points where the "significant change" scalar field is 1, 
    and writes the results to a new LAS file.

    Parameters:
    pathin (str): Path to the input LAS file.
    pathout (str): Path to the output LAS file.
    minpoints (int): Minimum number of points to form a dense region (minPts for DBSCAN).
    r (float): The maximum distance between two samples for one to be considered in the neighborhood of the other (epsilon for DBSCAN).
    """
    
    # Read the input LAS file
    with laspy.open(pathin) as las_file:
        las = las_file.read()

    # Extract XYZ coordinates and the "significant change" field
    points = np.vstack((las.x, las.y, las.z)).T

    try:
        significant_change = las['significant change']  # Replace with the correct field name if different
            # Filter points where "significant change" is 1
        points_filtered = points[significant_change == 1]

        if points_filtered.size == 0:
            print(f"No points with significant change in {pathin}")
            return
        flag = 0
    except:
        print(f"Error: 'significant_change' field not found in {pathin}")
        points_filtered = points
        flag = 1



    # Run DBSCAN clustering on filtered points
    dbscan = DBSCAN(eps=r, min_samples=minpoints)
    labels = dbscan.fit_predict(points_filtered)

    # Create a new array to hold cluster IDs for all points, set to -1 by default (meaning no cluster)
    cluster_ids = np.full(len(points), -1, dtype=np.int32)
    
    # Assign cluster IDs only to points that passed the "significant change" filter
    if not flag:
        cluster_ids[significant_change == 1] = labels

    # Add cluster labels as a new dimension to the LAS file
    las.add_extra_dim(laspy.ExtraBytesParams(name="ClusterID", type=np.int32))
    las.ClusterID = cluster_ids

    # Write the results to the output LAS file
    with laspy.open(pathout, mode='w', header=las.header) as las_output:
        las_output.write_points(las.points)

    print(f"DBSCAN clustering complete for significant change points. Results saved to {pathout}")


def process_erosion_files(base_dir, output_dir, minpoints, r):
    """
    Loops through subfolders in base_dir, finds all erosion2.las files, and runs DBSCAN on each.
    
    Parameters:
    base_dir (str): The path to the base directory containing m3c2_results subfolders.
    output_dir (str): The base path to the output directory where results will be stored.
    minpoints (int): Minimum number of points for DBSCAN.
    r (float): The maximum distance for DBSCAN.
    """
    
    # Loop through each subfolder in the m3c2_results directory
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.startswith('erosion2') and file.endswith('.las'):
                # Full path of the erosion2.las file
                erosion_file_path = os.path.join(root, file)
                # display(erosion_file_path)
                
                # Extract the subfolder name from the root path
                subfolder_name = Path(root).name
                # display(subfolder_name)

                # Define the output directory for this subfolder
                output_subfolder = os.path.join(output_dir, subfolder_name)
                # display(output_subfolder)

                # Create the output directory if it doesn't exist
                os.makedirs(output_subfolder, exist_ok=True)

                # Define the output LAS file path
                output_las_path = os.path.join(output_subfolder, f"{file[:-4]}_dbscan.las")
                # display(output_las_path)

                # Run DBSCAN on the erosion2.las file
                run_dbscan(erosion_file_path, output_las_path, minpoints, r)


# Example usage
base_dir = '/Volumes/group/LiDAR/LidarProcessing/changedetection_m3c2/m3c2_results'
output_dir = '/Volumes/group/LiDAR/LidarProcessing/changedetection_m3c2/grid_output'
minpoints = 20  # Minimum points for DBSCAN
r = .4  # Epsilon (maximum distance for DBSCAN)

process_erosion_files(base_dir, output_dir, minpoints, r)

DBSCAN clustering complete for significant change points. Results saved to /Volumes/group/LiDAR/LidarProcessing/changedetection_m3c2/grid_output/20190916_568_636_to_20190924_567_636/erosion2_dbscan.las
DBSCAN clustering complete for significant change points. Results saved to /Volumes/group/LiDAR/LidarProcessing/changedetection_m3c2/grid_output/20220701_567_770_to_20220706_567_770/erosion2_dbscan.las
DBSCAN clustering complete for significant change points. Results saved to /Volumes/group/LiDAR/LidarProcessing/changedetection_m3c2/grid_output/20190513_568_636_to_20190522_568_636/erosion2_dbscan.las
DBSCAN clustering complete for significant change points. Results saved to /Volumes/group/LiDAR/LidarProcessing/changedetection_m3c2/grid_output/20180115_590_622_to_20180116_590_622/erosion2_dbscan.las
DBSCAN clustering complete for significant change points. Results saved to /Volumes/group/LiDAR/LidarProcessing/changedetection_m3c2/grid_output/20190123_590_636_to_20190130_568_622/erosion2_d

In [ ]:
def makeGrid(pathin, pathout, polys, res):
    """
    Reads in a LAS file and a shapefile of polygons, calculates the mean of M3C2 distance 
    (absolute value) of points within each polygon for 1-meter height bins, and writes the result to a CSV file.

    Also plots the grid with Polygon ID on the x-axis and Z value on the y-axis, even for bins with no points.

    Parameters:
    pathin (str): Path to the input LAS file.
    pathout (str): Path to the output CSV file.
    polys (str): Path to the shapefile of polygons.
    """
    
    # Read the input LAS file
    with laspy.open(pathin) as las_file:
        las = las_file.read()

    # Extract XYZ coordinates, M3C2 distances, and ClusterID (assumed stored as extra fields)
    points = np.vstack((las.x, las.y, las.z, las['M3C2 distance'], las['ClusterID'])).T

    # Filter points where ClusterID is NOT -1
    points = points[points[:, 4] != -1]

    # Take absolute value of M3C2 Distance
    points[:, 3] = np.abs(points[:, 3])

    # Read the shapefile containing the polygons
    polygons = gpd.read_file(polys)
    # display(polygons)

    # Create a GeoDataFrame from the LAS points
    gdf_points = gpd.GeoDataFrame(
        points, 
        columns=['X', 'Y', 'Z', 'M3C2 Distance', 'ClusterID'],
        geometry=[Point(x, y) for x, y in zip(points[:, 0], points[:, 1])]
    )

    # Initialize a list to hold results for each polygon
    results = []

    # vertical bins
    z_bins = np.arange(0, 30, res)

    # Loop through each polygon and calculate the mean M3C2 distance for 1-meter Z-bins
    for i, polygon in polygons.iterrows():
        points_in_polygon = gdf_points[gdf_points.within(polygon.geometry)]

        # Define bins for height (Z-axis) from 0 to max height (e.g., 35 meters)
        # z_bins = np.arange(0, 30, 1)  # Create bins from 0 to 35 meters in 1-meter increments

        # Initialize a dictionary to store results for each polygon and each Z-bin
        polygon_result = {'Polygon_ID': polygon.index}
        # clusterID

        # Loop through each Z-bin (1-meter horizontal slice)
        for z in range(len(z_bins) - 1):
            # Select points within this 1-meter horizontal bin (Z-axis)
            points_in_bin = points_in_polygon[
                (points_in_polygon['Z'] >= z_bins[z]) & (points_in_polygon['Z'] < z_bins[z+1])
            ]
            
            # clusterID = points_in_bin['ClusterID'].mode().values[0] 

            if len(points_in_bin) > 0:
                # Compute the mean absolute M3C2 distance for this Z-bin
                mean_m3c2_in_bin = points_in_bin['M3C2 Distance'].median()
                cluster_id = points_in_bin['ClusterID'].mode().values[0]
            else:
                # If no points in this bin, set the mean distance to NaN
                mean_m3c2_in_bin = np.nan
                cluster_id = np.nan

            # Add the mean distance for this bin to the results dictionary
            polygon_result[f'M3C2_{z}m'] = [mean_m3c2_in_bin, cluster_id]
            # polygon_result[f'ClusterID_{z}m'] = cluster_id

        # Append the results for this polygon to the main results list
        results.append(polygon_result)

    # Create a DataFrame to store the results
    df_results = pd.DataFrame(results)
    # display(df_results)

    # Write the results to a CSV file
    df_results.to_csv(pathout, index=False)
    print(f"Results written to {pathout}")

    return df_results



In [ ]:
polys = '/Users/cjmack/Documents/Papers/Cliffs/Code/Polygones/PolygonesDelMar1m.shp'
pathin = '/Users/cjmack/Documents/Papers/Cliffs/Scratch/erosion2_dbscan.las'
pathout = '/Users/cjmack/Documents/Papers/Cliffs/Scratch/Test.csv'
resolution = .1

output = makeGrid(pathin, pathout, polys, resolution)